In [ ]:
# !pip install stackview cellpose 'napari[all]' ultrack cucim ipycanvas==0.11
# !pip install git+https://github.com/Janelia-Trackathon-2023/traccuracy
COLAB = True
# COLAB = False

In [ ]:
!wget -nc http://data.celltrackingchallenge.net/training-datasets/Fluo-C2DL-Huh7.zip
!unzip -n Fluo-C2DL-Huh7.zip

In [ ]:
from pathlib import Path

import numpy as np
import stackview
from dask.array.image import imread
from numpy.typing import ArrayLike
from rich import print

from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics

from ultrack import track, to_tracks_layer, tracks_to_zarr, to_ctc
from ultrack.utils import labels_to_edges
from ultrack.config import MainConfig
from ultrack.imgproc import normalize
from ultrack.imgproc.segmentation import Cellpose
from ultrack.utils.array import array_apply

In [ ]:
if COLAB:
    viewer = None

    # fixes colab encoding error
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"

    # enabling colab output
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except ModuleNotFoundError as e:
        print(e)
else:
    import napari
    viewer = napari.Viewer()

In [ ]:
dataset = "02"
path = Path("Fluo-C2DL-Huh7") / dataset
image = imread(str(path / "*.tif"))

if COLAB:
    display(stackview.slice(image))
else:
    viewer.add_image(image)

In [ ]:
cellpose = Cellpose(model_type="cyto2", gpu=True)

def predict(frame: ArrayLike, gamma: float) -> ArrayLike:
    norm_frame = normalize(np.asarray(frame), gamma=gamma)
    return cellpose(norm_frame, tile=False, normalize=False, diameter=75.0)

In [ ]:
cellpose_labels = np.zeros(image.shape, dtype=np.int32)
array_apply(
    image,
    out_array=cellpose_labels,
    func=predict,
    gamma=0.5,
)

if COLAB:
    display(stackview.curtain(image, cellpose_labels))
else:
    viewer.add_labels(cellpose_labels)

In [ ]:
detection, contours = labels_to_edges(cellpose_labels, sigma=5.0)

In [ ]:
if COLAB:
    display(stackview.curtain(image, detection))
else:
    viewer.add_labels(detection)

In [ ]:
if COLAB:
    display(stackview.curtain(image, contours))
else:
    viewer.add_image(contours)

In [ ]:
config = MainConfig()

# Candidate segmentation parameters
config.segmentation_config.n_workers = 4
config.segmentation_config.min_area = 2500
config.segmentation_config.min_frontier = 0.1

# Setting the maximum number of candidate neighbors and maximum spatial distance between cells
config.linking_config.max_neighbors = 5
config.linking_config.max_distance = 100
config.linking_config.n_workers = 4

# Adding absurd weight to division because there are few dividing cells
config.tracking_config.division_weight = -100

# Very few tracks enter/leave the field of view, increasing penalization
config.tracking_config.disappear_weight = -1
config.tracking_config.appear_weight = -1

print(config)

In [ ]:
track(
    config,
    detection=detection,
    edges=contours,
    overwrite=True
)

In [ ]:
tracks_df, graph = to_tracks_layer(config)
tracks_df.to_csv(f"{dataset}_tracks.csv", index=False)

segments = tracks_to_zarr(
    config,
    tracks_df,
    overwrite=True,
)

if COLAB:
    display(stackview.curtain(image, segments))
else:
    viewer.add_tracks(
        tracks_df[["track_id", "t", "y", "x"]],
        name="tracks",
        graph=graph,
        visible=True,
    )

    viewer.add_labels(segments, name="segments").contour = 2

In [ ]:
name = f"{path.parent.name}_{path.name}".upper()
output_path = Path(name) / "TRA"
to_ctc(output_path, config, overwrite=True)

gt_path = path.parent / f"{dataset}_GT" / "TRA"

run_metrics(
    gt_data=load_ctc_data(gt_path),
    pred_data=load_ctc_data(output_path),
    matcher=CTCMatched,
    metrics=[CTCMetrics],
)["CTCMetrics"]